In [157]:
import pandas as pd
import altair as alt
import numpy as np
from vega_datasets import data as dataset
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [158]:
#Opening spreadsheet and replacing missing values with NaN
excel = pd.read_excel("sheets/life_index.xlsx")
excel = excel.replace('..', np.nan)
excel = excel.replace(' ', '')

In [159]:
#Imputing missing values

imputer = IterativeImputer(max_iter=15, random_state=0)

# Gets only numerical data
excel_copy = excel.iloc[:,1:].copy()
excel_copy_mask = excel_copy.isna()
imputed_valuess = imputer.fit_transform(excel_copy)
excel_imputed = pd.DataFrame(data=imputed_valuess)
excel_imputed = excel_imputed.abs()

# Gets columns names except for first one. Did this because had to discard country names before imputing them.
excel_names = excel.iloc[:0,1:].copy()

# transforms dataframe into 1 dimensional format
excel_names = excel_names.columns
# gets first column with all rows
countries = excel.iloc[:,:1]
# creates new dataframe with imputted data and column names
final_imputed = pd.DataFrame(data=excel_imputed.values,columns=excel_names)
# inserts country names
final_imputed.insert(loc=0, column='Countries',value=countries)

final_imputed.to_excel("Table_Imputed.xlsx")

In [160]:
to_normalize = excel_imputed.copy()
df_pos = to_normalize [[2,3,4,5,7,8,9,10,11,13,15,17,18,19,20,23]].copy()
df_neg = to_normalize[[0,1,6,12,14,16,21,22]].copy()

normalized_df_pos = df_pos.copy()
normalized_df_neg = df_neg.copy()
normalized_df_pos = (normalized_df_pos - normalized_df_pos.min())/(normalized_df_pos.max() - normalized_df_pos.min()) * 100
normalized_df_neg = (1 - (normalized_df_neg - normalized_df_neg.min())/(normalized_df_neg.max() - normalized_df_neg.min())) * 100
normalized_result = pd.concat([normalized_df_neg, normalized_df_pos],axis=1).round(2)
normalized_result = normalized_result[sorted(normalized_result.columns)]
format_result = pd.DataFrame(data=normalized_result.values,columns=excel_names)
# format_result
format_result.insert(loc=0, column='Countries',value=countries)
format_result.to_json("full_scores.json")


In [161]:
# I need 2 tables, one with full score for each country and another for scores of each category.

In [162]:
full_score = format_result.sum(axis='columns')
full_score = pd.DataFrame(full_score)
full_score.insert(loc=0, column='Countries',value=countries)
full_score.rename(columns={0:'Score'}, inplace=True)


C:\Users\dovyd\AppData\Local\Temp\ipykernel_21696\17844380.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  full_score = format_result.sum(axis='columns')


In [163]:
#Category Scores

housing = format_result.iloc[:,:3].sum(axis='columns').div(3)
education = format_result.iloc[:,3:6].sum(axis='columns').div(3)
environment = format_result.iloc[:,6:8].sum(axis='columns').div(2)
health = format_result.iloc[:,8:10].sum(axis='columns').div(2)
income = format_result.iloc[:,10:12].sum(axis='columns').div(2)
jobs = format_result.iloc[:,12:16].sum(axis='columns').div(4)
worklife = format_result.iloc[:,16:18].sum(axis='columns').div(2)
community = format_result.iloc[:,18:19]
civic_engagement = format_result.iloc[:,19:21].sum(axis='columns').div(2)
safety = format_result.iloc[:,21:23].sum(axis='columns').div(2)
life_satisfaction = format_result.iloc[:,23:24]

result = pd.concat([housing,education,environment,health,income,jobs,worklife,community,civic_engagement,safety,life_satisfaction],axis=1)
result.rename(columns={0:'Housing',1:'Education',2:'Environment',3:'Health',4:'Income',5:'Jobs',6:'Worklife','Quality of support network':'Community',7:'Civic Engagement',8:'Safety',}, inplace=True)
result.insert(loc=0, column='Countries',value=countries)

C:\Users\dovyd\AppData\Local\Temp\ipykernel_21696\3809542928.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  housing = format_result.iloc[:,:3].sum(axis='columns').div(3)


In [164]:
import geopandas as gpd

In [165]:
url = "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"
countries_shape = gpd.read_file(url) # zipped shapefile
countries_shape = countries_shape[['NAME', 'CONTINENT', 'ISO_A3', 'geometry']]
countries_shape.loc[21,'ISO_A3'] = 'NOR'
countries_shape.loc[43,'ISO_A3'] = 'FRA'

In [166]:
#Choropleth View


#Importing Tables
data_categories = pd.read_excel("sheets/out_of_a_100.xlsx");
choro_map = alt.Legend(titleAlign='left', legendX=0, legendY=25, orient='none')
#Parameters
total_select = alt.selection_point(fields=["Country"])
total_select_empty = alt.selection_point(fields=["Country"], empty = False)
countries_plot = list(data_categories.ISO)
color_lsoa = alt.condition(total_select, alt.Color("Total:Q", 
             scale = alt.Scale(scheme="spectral",domain=[0, 100], clamp=True),
             title="Well-being Score", legend = choro_map),
             alt.value("lightgray")) 

#World Map                                                
sphere = alt.sphere()
graticule = alt.graticule()

basemap = (alt.layer(
alt.Chart(sphere).mark_geoshape(fill='white'),
alt.Chart(graticule).mark_geoshape(stroke='lightgray'),
alt.Chart(countries_shape).mark_geoshape(fill='lightgray')))

choro_title = alt.TitleParams(text='Better Life Index Across the Globe', 
                              subtitle=['The Index Represents a Score Calculated from 23 Subcategories of Well-being. Greyed-out Countries Symbolise Missing Data.','Shift + Click to Select Multiple Countries. Hover For Details.'],
                              subtitleFontSize=14, fontSize=20, anchor='start', offset=-15, color='#333333', subtitleColor='#555555')

#Choropleth Map
choro_lsoa = (alt.Chart(countries_shape, title=choro_title).mark_geoshape(stroke='#979797'
    ).transform_lookup(
        lookup="ISO_A3",
        from_=alt.LookupData(
            data=data_categories, key='ISO', fields=['Total','Country','Rank'])
            ).transform_filter(alt.FieldOneOfPredicate(field="ISO_A3", oneOf=countries_plot)
            ).encode(
                color=color_lsoa,
                tooltip=[alt.Tooltip("Country:N", title="Country"), 
                         alt.Tooltip("Total:Q", title="Well-being Index", format='.1f'), 
                         alt.Tooltip("Rank:Q", title="Global Ranking")]).add_params(total_select, total_select_empty))

final_map = ((basemap + choro_lsoa).project("equalEarth").properties(width=700, height=400))

In [167]:
#Bar Chart View

melted_data = pd.melt(data_categories, id_vars=['ISO','Country'], var_name='column', value_name='y')

column_selection = alt.selection_point(
    name='Select',
    fields=['column'],
    bind=alt.binding_select(options=data_categories.columns[3:].tolist()),
    value='Housing'
)
bar_title = alt.TitleParams(text='Comparison of Well-being By Category', 
                            subtitle=['Compare the 11 Categories That Make Up the Better Life Index as Well as the Total Scores.','Use Dropdown To Select Category. Hover For Detail.'],
                            subtitleFontSize=14, fontSize=20, anchor='start', offset=10, dx=75, color='#333333', subtitleColor='#414141')

#Selective Bar Chart
bar_chart = (alt.Chart(melted_data, title=bar_title).mark_bar(color="#0000FF").encode(
    y=alt.Y("Country:N",title=''),
    color=alt.Color('Country', title="Country"),
    x=alt.X("y", title="Score for Selected Category")).transform_filter(column_selection).transform_filter(total_select_empty).add_params(column_selection).properties(width=675, height=225))

average_life_index = pd.read_excel("sheets/average_categories.xlsx")
melted_average = pd.melt(average_life_index, id_vars=['Country'], var_name='column', value_name='y')

# Bar that displays the world average.
average_index_chart = (alt.Chart(melted_average).mark_bar(color="#0000FF").encode(
    color=alt.Color('Country', title=''),
    x=alt.X("y",axis=alt.Axis(tickMinStep=1)).scale(domain=(0, 100)),
    y=alt.Y("Country:N")).properties(width=675, height=225)).transform_filter(column_selection)

final_bar_chart = (bar_chart + average_index_chart).encode(tooltip=[alt.Tooltip("Country:N", title="Country"), alt.Tooltip("y:Q",title='Score', format='.1f')])

MCV = alt.vconcat(final_map,final_bar_chart)



In [168]:
#HeatMap View

quality = pd.concat([life_satisfaction,health,environment], axis=1)
quality.rename(columns={0:'health',1:'environment'}, inplace=True)
correlation_matrix = format_result.corr()
correlation_melted = correlation_matrix.reset_index().melt('index')
correlation_melted.columns = ['variable1', 'variable2', 'correlation']

heatmap_title = alt.TitleParams(text='Correlation Map of Well-being Indicators', 
                                subtitle=['Negative Correlation means that if one value increases, the other value decreases.',
                                          'Positive Correlation means that if one value increases, the other value also increases.',
                                          'Hover for More Details']
                                ,subtitleFontSize=14, fontSize=20, anchor='start', offset=10, dx=185, color='#333333', subtitleColor='#414141')

heatmap_legend = alt.Legend(titleAlign='left', legendX=1450, legendY=200, orient='none')
heatmap = (
    alt.Chart(correlation_melted,width=450, height=450, title=heatmap_title)
    .mark_rect()
    .encode(
        x=alt.X('variable1:O',axis=alt.Axis(labelAngle=315, title=None)),
        y=alt.Y('variable2:O', axis=alt.Axis(title=None)),
        color=alt.Color('correlation:Q',title="Correlation",scale=alt.Scale(scheme="blueorange", domain=[-1,1]),legend=heatmap_legend),
        tooltip=[alt.Tooltip("correlation:Q", title="Correlation Index", format='.2f'),
                 alt.Tooltip("variable1:N", title="Indicator 1"),
                 alt.Tooltip("variable2:N", title="Indicator 2")]
    )
)

C:\Users\dovyd\AppData\Local\Temp\ipykernel_21696\3476027020.py:5: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlation_matrix = format_result.corr()


In [169]:
export = alt.hconcat(alt.vconcat(MCV), heatmap, center=True
                    ).properties(padding=5, 
                    title=alt.TitleParams(text="OECD and Partner Country Wellbeing & Correlation of Wellbeing Factors", 
                    fontSize=28 ,anchor='middle', align='right', offset=30, dx=237, color='#111111'), center=True)

export.save('chart.html')